In [ ]:
!pip install pytube moviepy openai whisper

In [ ]:
from pytube import YouTube
from moviepy.editor import *
import whisper
import os

def download_video(url):
    yt = YouTube(url)
    stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    return stream.download()

def split_video(video_path, chunk_length=24*60):
    video = VideoFileClip(video_path)
    duration = video.duration
    chunks = []
    for start in range(0, int(duration), chunk_length):
        end = min(start + chunk_length, duration)
        chunk = video.subclip(start, end)
        chunk_filename = f'{video_path}_chunk_{start}_{end}.mp4'
        chunk.write_videofile(chunk_filename, codec='libx264')
        chunks.append(chunk_filename)
    video.close()
    return chunks

def transcribe_chunks(chunks):
    model = whisper.load_model('large')
    for chunk in chunks:
        result = model.transcribe(chunk)
        with open(f'{chunk}.txt', 'w') as f:
            f.write(result['text'])
        os.remove(chunk)

url = 'YOUR_YOUTUBE_VIDEO_URL_HERE'
video_path = download_video(url)
chunks = split_video(video_path)
transcribe_chunks(chunks)